<a href="https://colab.research.google.com/github/xander-bennett/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/Copy_of_Mongodb_assingment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and setup

In [0]:
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
import pymongo
from pymongo import UpdateOne
import sqlite3
import pandas as pd

In [0]:
# If working on colab:
# Uncomment the next lines and supply your own mongodb connection string on the next cell
# !wget https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3

In [0]:
# connect to mongodb
client = MongoClient('mongodb://mongo:27017/')
mydb = client['rpg']
char_record = mydb['character']

In [0]:
# sqlite connection
con = sqlite3.connect('rpg_db.sqlite3')
cur = con.cursor()
classes = [
    'charactercreator_cleric',
    'charactercreator_fighter',
    'charactercreator_mage',
    'charactercreator_thief',
    'charactercreator_necromancer'
]

# Because pandas is really convinient
def sql_df(query, con=con):
    '''
    Takes an sql query string and a db connection object and outputs
    a pandas dataframe from the results
    '''
    return pd.read_sql(query, con)

## Insert Character data from the charactercreator_character table to our mongo database

In [0]:

char_record.create_index([('character_id', pymongo.ASCENDING)], unique=True)
characters = sql_df('SELECT * FROM charactercreator_character').to_dict(orient='records')
# Load data if not loaded already
try:
    char_record.insert_many(characters)
except BulkWriteError as e:
        print('Record already written')

### Compare Entries (chracter_id: 1)

In [0]:
sql_df('SELECT * FROM charactercreator_character WHERE character_id = 1')

,character_id,name,level,exp,hp,strength,intelligence,dexterity,wisdom
0,1,Aliquid iste optio reiciendi,0,0,10,1,1,1,1


In [0]:
char_record.find_one({'character_id': 1})

{'_id': ObjectId('5dc3bd315d4910a3230f05a5'),
 'character_id': 1,
 'name': 'Aliquid iste optio reiciendi',
 'level': 0,
 'exp': 0,
 'hp': 10,
 'strength': 1,
 'intelligence': 1,
 'dexterity': 1,
 'wisdom': 1}

### Insert Character data from other tables

In [0]:
# RPG Classes
base_classes = [
    'charactercreator_cleric',
    'charactercreator_fighter',
    'charactercreator_mage',
    'charactercreator_thief',
    'charactercreator_necromancer'
]
# Merge the class data to character record

# Track char classes
char_classes = {}
# List of operations for bulk writing
operations = []
for cls in base_classes:
    for rec in sql_df(f'SELECT * FROM {cls}').to_dict(orient='records'):
        # Handle special case
        # Necrmonacer is a subclass of mage
        char_foreign_key = 'mage_ptr_id' if cls == 'charactercreator_necromancer'\
        else 'character_ptr_id'
        character_id = rec.pop(char_foreign_key)
        #use the last word for class name
        class_name = cls.split('_')[-1]
        rec[class_name] = True
        char_classes.setdefault(character_id,[]).append(class_name)
        rec['classes'] = char_classes[character_id]
        rec['items'] = []
        query = {'character_id': character_id}
        operations.append(UpdateOne(query, {'$set': rec}, upsert=True))
# Execute bulk operations
char_record.bulk_write(operations)

### Compare!

In [0]:
# How do we know which class/es a character belongs to?
# Lots of joins!
base_query = 'SELECT * FROM charactercreator_character as cc'
def join_query(tablename):
    char_foreign_key = 'mage_ptr_id' if tablename == 'charactercreator_necromancer'\
    else 'character_ptr_id'
    class_name = tablename.split('_')[-1]
    return f'''
        LEFT JOIN (SELECT *, True AS {class_name} FROM  {tablename}) {class_name}\
        ON cc.character_id = {class_name}.{char_foreign_key}
        '''
sql_df(
    base_query + ' '.join([join_query(x) for x in classes])+
    'WHERE cc.character_id=1'
).dropna(axis=1)


,character_id,name,level,exp,hp,strength,intelligence,dexterity,wisdom,character_ptr_id,using_shield,rage,fighter
0,1,Aliquid iste optio reiciendi,0,0,10,1,1,1,1,1,0,100,1


In [0]:
char_record.find_one({'character_id': 1})

{'_id': ObjectId('5dc3bd315d4910a3230f05a5'),
 'character_id': 1,
 'name': 'Aliquid iste optio reiciendi',
 'level': 0,
 'exp': 0,
 'hp': 10,
 'strength': 1,
 'intelligence': 1,
 'dexterity': 1,
 'wisdom': 1,
 'classes': ['fighter'],
 'fighter': True,
 'items': [],
 'rage': 100,
 'using_shield': 0}

In [0]:
# A characters from different classes have different sets valid atributes
sql_df(
    base_query + ' '.join([join_query(x) for x in classes])+
    'WHERE necromancer=True LIMIT 1'
).dropna(axis=1)


,character_id,name,level,exp,hp,strength,intelligence,dexterity,wisdom,character_ptr_id,has_pet,mana,mage,mage_ptr_id,talisman_charged,necromancer
0,292,Vero a,0,0,10,1,1,1,1,292,1,100,1,292,1,1


In [0]:
char_record.find_one({'classes': 'necromancer'})

{'_id': ObjectId('5dc3bd315d4910a3230f06c8'),
 'character_id': 292,
 'name': 'Vero a',
 'level': 0,
 'exp': 0,
 'hp': 10,
 'strength': 1,
 'intelligence': 1,
 'dexterity': 1,
 'wisdom': 1,
 'classes': ['mage', 'necromancer'],
 'has_pet': 1,
 'items': [],
 'mage': True,
 'mana': 100,
 'necromancer': True,
 'talisman_charged': 1}

In [0]:
# Add items
operations = []
from math import isnan
for rec in sql_df('''
        SELECT character_id, ai.item_id, name, value, weight, power, weapon
        FROM charactercreator_character_inventory AS ci 
        JOIN armory_item AS ai ON ci.item_id=ai.item_id
        LEFT JOIN (SELECT *, True as weapon FROM armory_weapon) 
        AS aw on aw.item_ptr_id = ai.item_id
        ORDER BY ai.item_id
    ''').to_dict(orient='records'):
    character_id = rec.pop('character_id')
    keys = list(rec.keys())
    # Remove nan's as fields
    # joining the weapons introduced a lot of nan's to remove
    for key in keys:
        v = rec[key]     
        if type(v) == float and isnan(v):
            rec.pop(key)
    if rec.get('weapon') == 1:
        rec['weapon'] = True
    query = {'character_id': character_id}
    operations.append(UpdateOne(query, {'$addToSet': {'items': rec}}, upsert=True))
char_record.bulk_write(operations)


In [0]:
# Show items of character 1
sql_df('''
    SELECT character_id, ai.item_id, name, value, weight, power, weapon
    FROM charactercreator_character_inventory AS ci 
    JOIN armory_item AS ai ON ci.item_id=ai.item_id
    LEFT JOIN (SELECT *, True as weapon FROM armory_weapon) 
    AS aw on aw.item_ptr_id = ai.item_id
    WHERE ci.character_id=1
    ORDER BY ai.item_id
    ''').dropna(1)

,character_id,item_id,name,value,weight
0,1,20,Dolores rem v,0,0
1,1,58,Quos re,0,0
2,1,85,Alias laboriosam sapiente e,0,0


In [0]:
char_record.find_one({'character_id': 1})

{'_id': ObjectId('5dc3bd315d4910a3230f05a5'),
 'character_id': 1,
 'name': 'Aliquid iste optio reiciendi',
 'level': 0,
 'exp': 0,
 'hp': 10,
 'strength': 1,
 'intelligence': 1,
 'dexterity': 1,
 'wisdom': 1,
 'classes': ['fighter'],
 'fighter': True,
 'items': [{'item_id': 20, 'name': 'Dolores rem v', 'value': 0, 'weight': 0},
  {'item_id': 58, 'name': 'Quos re', 'value': 0, 'weight': 0},
  {'item_id': 85,
   'name': 'Alias laboriosam sapiente e',
   'value': 0,
   'weight': 0}],
 'rage': 100,
 'using_shield': 0}

In [0]:
# Show items of character 5 (someone with weapon)
sql_df('''
    SELECT character_id, ai.item_id, name, value, weight, power, weapon
    FROM charactercreator_character_inventory AS ci 
    JOIN armory_item AS ai ON ci.item_id=ai.item_id
    LEFT JOIN (SELECT *, True as weapon FROM armory_weapon) 
    AS aw on aw.item_ptr_id = ai.item_id
    WHERE character_id=5
    ORDER BY ai.item_id
    ''')


,character_id,item_id,name,value,weight,power,weapon
0,5,7,Omnis,0,0,NaN,NaN
1,5,96,Commodi deserunt in illo,0,0,NaN,NaN
2,5,140,Quos nihil quibusdam,0,0,0.0,1.0
3,5,145,Corporis fug,0,0,0.0,1.0


In [0]:
char_record.find_one({'character_id': 5})

{'_id': ObjectId('5dc3bd315d4910a3230f05a9'),
 'character_id': 5,
 'name': 'At id recusandae expl',
 'level': 0,
 'exp': 0,
 'hp': 10,
 'strength': 1,
 'intelligence': 1,
 'dexterity': 1,
 'wisdom': 1,
 'classes': ['fighter'],
 'fighter': True,
 'items': [{'item_id': 7, 'name': 'Omnis', 'value': 0, 'weight': 0},
  {'item_id': 96, 'name': 'Commodi deserunt in illo', 'value': 0, 'weight': 0},
  {'item_id': 140,
   'name': 'Quos nihil quibusdam',
   'value': 0,
   'weight': 0,
   'power': 0.0,
   'weapon': True},
  {'item_id': 145,
   'name': 'Corporis fug',
   'value': 0,
   'weight': 0,
   'power': 0.0,
   'weapon': True}],
 'rage': 100,
 'using_shield': 0}

In [0]:
char_record.find_one({'character_id': 5})['items']

[{'item_id': 7, 'name': 'Omnis', 'value': 0, 'weight': 0},
 {'item_id': 96, 'name': 'Commodi deserunt in illo', 'value': 0, 'weight': 0},
 {'item_id': 140,
  'name': 'Quos nihil quibusdam',
  'value': 0,
  'weight': 0,
  'power': 0.0,
  'weapon': True},
 {'item_id': 145,
  'name': 'Corporis fug',
  'value': 0,
  'weight': 0,
  'power': 0.0,
  'weapon': True}]

# Queries!

## How many characters are there in total


In [0]:
char_record.count_documents({})


302

## How many characters per class


In [0]:
# Multiple queries
for cls in base_classes:
    cls_name = cls.split('_')[-1]
    cnt = char_record.count_documents({cls_name: True})
    print(f'{cls_name} count: {cnt}')

cleric count: 75
fighter count: 68
mage count: 108
thief count: 51
necromancer count: 11


In [0]:
# Single query
# You may also write a query builder to make the code 
# shorter/less repetitive
pipeline = [
    {'$project': {'character_id': 1, 'cleric':
                    {'$cond': [ { '$eq': ["$cleric", True] }, 1, 0]},
                    'fighter': 
                    {'$cond': [ { '$eq': ["$fighter", True] }, 1, 0]},
                    'mage': 
                    {'$cond': [ { '$eq': ["$mage", True] }, 1, 0]},
                    'thief': 
                    {'$cond': [ { '$eq': ["$thief", True] }, 1, 0]},
                    'necromancer': 
                    {'$cond': [ { '$eq': ["$necromancer", True] }, 1, 0]}}},
        
    {'$group': {'_id': '', 'cleric': {'$sum':'$cleric'},
                'fighter': {'$sum':'$fighter'},
                'mage': {'$sum':'$mage'},
                'thief': {'$sum':'$thief'},
                'necromancer': {'$sum':'$necromancer'} }},
    {'$sort': {'_id':1}},
]
list(char_record.aggregate(pipeline))

[{'_id': '',
  'cleric': 75,
  'fighter': 68,
  'mage': 108,
  'thief': 51,
  'necromancer': 11}]

## How many of the Items are weapons? How many are not?


In [0]:
pipeline = [
    {'$unwind': '$items'},
    {'$match': {'items.weapon': True}},
    {'$count': "Total weapons"}
]
list(char_record.aggregate(pipeline))

[{'Total weapons': 203}]

In [0]:
pipeline = [
    {'$unwind': '$items'},
    {'$match': {'items.weapon': {'$ne':True}}},
    {'$count': "Total Non-weapons"}
]
list(char_record.aggregate(pipeline))

[{'Total Non-weapons': 695}]

## How many Items does each character have? (Return first 20 rows)


In [0]:
pipeline = [
    {'$project': {'_id': '$character_id', 'name': '$name', 'item_count': {'$size': '$items'}}},
    { '$limit': 20 }
]
list(char_record.aggregate(pipeline))

[{'_id': 1, 'name': 'Aliquid iste optio reiciendi', 'item_count': 3},
 {'_id': 2, 'name': 'Optio dolorem ex a', 'item_count': 3},
 {'_id': 3, 'name': 'Minus c', 'item_count': 2},
 {'_id': 4, 'name': 'Sit ut repr', 'item_count': 4},
 {'_id': 5, 'name': 'At id recusandae expl', 'item_count': 4},
 {'_id': 6, 'name': 'Non nobis et of', 'item_count': 1},
 {'_id': 7, 'name': 'Perferendis', 'item_count': 5},
 {'_id': 8, 'name': 'Accusantium amet quidem eve', 'item_count': 3},
 {'_id': 9, 'name': 'Sed nostrum inventore error m', 'item_count': 4},
 {'_id': 10, 'name': 'Harum repellendus omnis od', 'item_count': 4},
 {'_id': 11, 'name': 'Itaque ut commodi,', 'item_count': 3},
 {'_id': 12, 'name': 'Molestiae quis', 'item_count': 3},
 {'_id': 13, 'name': 'Ali', 'item_count': 4},
 {'_id': 14, 'name': 'Tempora quod optio possimus il', 'item_count': 4},
 {'_id': 15, 'name': 'Sed itaque beatae pari', 'item_count': 4},
 {'_id': 16, 'name': 'Quam dolor', 'item_count': 1},
 {'_id': 17, 'name': 'Molestias

## How many Weapons does each character have? (Return first 20 rows)¶


In [0]:
pipeline = [
    {'$unwind': '$items'},
    {'$project': {'character_id': 1, 'name': '$name', 'weapons': 
                  {'$cond': [ { '$eq': ["$items.weapon", True] }, 1, 0]}}},
    {'$group': {'_id': '$character_id', 'name': {'$first': '$name'},  
                'weapons_count': {'$sum':'$weapons'} }},
    {'$sort': {'_id':1}},
    { '$limit': 20 }
]
list(char_record.aggregate(pipeline))

[{'_id': 1, 'name': 'Aliquid iste optio reiciendi', 'weapons_count': 0},
 {'_id': 2, 'name': 'Optio dolorem ex a', 'weapons_count': 0},
 {'_id': 3, 'name': 'Minus c', 'weapons_count': 0},
 {'_id': 4, 'name': 'Sit ut repr', 'weapons_count': 0},
 {'_id': 5, 'name': 'At id recusandae expl', 'weapons_count': 2},
 {'_id': 6, 'name': 'Non nobis et of', 'weapons_count': 0},
 {'_id': 7, 'name': 'Perferendis', 'weapons_count': 1},
 {'_id': 8, 'name': 'Accusantium amet quidem eve', 'weapons_count': 0},
 {'_id': 9, 'name': 'Sed nostrum inventore error m', 'weapons_count': 0},
 {'_id': 10, 'name': 'Harum repellendus omnis od', 'weapons_count': 0},
 {'_id': 11, 'name': 'Itaque ut commodi,', 'weapons_count': 1},
 {'_id': 12, 'name': 'Molestiae quis', 'weapons_count': 0},
 {'_id': 13, 'name': 'Ali', 'weapons_count': 0},
 {'_id': 14, 'name': 'Tempora quod optio possimus il', 'weapons_count': 0},
 {'_id': 15, 'name': 'Sed itaque beatae pari', 'weapons_count': 0},
 {'_id': 16, 'name': 'Quam dolor', 'wea

## On average, how many items does each Character have?

In [0]:
pipeline = [
    {'$unwind': '$items'},
    {'$group': {'_id': '$character_id', 'count':{'$sum':1}}},
    {'$group': {'_id':'', 'Average Items':{'$avg':'$count'}}},
]
list(char_record.aggregate(pipeline))

[{'_id': '', 'Average Items': 2.9735099337748343}]

## On average, how many weapons does each Character have?

In [0]:
pipeline = [
    {'$unwind': '$items'},
    {'$project': {'character_id': 1, 'weapons': 
                  {'$cond': [ { '$eq': ["$items.weapon", True] }, 1, 0]}}},
    {'$group': {'_id': '$character_id', 'weapons': {'$sum':'$weapons'} }},
    {'$group': {'_id':'', 'Average weapons':{'$avg':'$weapons'}}},
    
]
list(char_record.aggregate(pipeline))

[{'_id': '', 'Average weapons': 0.6721854304635762}]